# Patch labeling

1. Weakly Supervised Learning

In [16]:
!pwd

/workspace/ToxRepresentatonCNN/ipynb


In [1]:
import sys

root = ".."
sys.path.append(root)

In [15]:
import yaml
import os
from toxreprcnn.train import train_loop
from toxreprcnn.utils import Logger
from toxreprcnn.dataset import load_features
from toxreprcnn.data_split import Fold, CrossValidation
from toxreprcnn.evaluate import (
    Metrics,
    macro_accuracy,
    macro_auroc,
    macro_balanced_accuracy,
    auroc,
    accuracy,
    macro_r2_score,
)
import pickle
import torch
import torch.nn as nn
import timm
from toxreprcnn.utils import fix_seed
import pandas as pd
import albumentations as albu
from albumentations.pytorch import ToTensorV2
from toxreprcnn.dataset import ToxReprCNNDataset, ToxReprCNNBalancedSampler
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import GroupKFold

In [ ]:
tile_dir = f"{root}/TGGATEs/tiles"
save_path = f"{root}/output/wsl/result"
random_state = 123
model_name = "tf_efficientnet_b4_ns"
feature_extraction = False
pretrained = True
image_size = 512
n_tiles = 100
num_workers = 4
train_batch_size = 32
valid_batch_size = 32
learning_rate = 0.0005
n_epochs = 20
verbose = 20
device = "cuda"
debug = False

In [10]:
info_path = f"{root}/data/TGGATEs/processed/info.csv"
info = pd.read_csv(info_path)

info["EG"] = 100 * info["EXP_ID"] + info["GROUP_ID"]

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
torch.backends.cudnn.benchmark = True
random_state = 123
fix_seed(random_state)

In [17]:
# gkf = GroupKFold(n_splits=5)

# metrics = [
#     Metrics("macro AUROC", macro_auroc, "+", limit=(0, 1)),
#     Metrics("macro AUROC", macro_accuracy, "+", limit=(0, 1)),
# ]


# train_transform = [
#     albu.RandomCrop(image_size, image_size),
#     albu.HorizontalFlip(p=0.5),
#     albu.VerticalFlip(p=0.5),
#     albu.Normalize(
#         mean=[0.485, 0.456, 0.406],
#         std=[0.229, 0.224, 0.225],
#     ),
#     ToTensorV2(),
# ]
# train_transform = albu.Compose(train_transform)
# valid_transform = albu.Compose(
#     [
#         albu.CenterCrop(image_size, image_size),
#         albu.Normalize(
#             mean=[0.485, 0.456, 0.406],
#             std=[0.229, 0.224, 0.225],
#         ),
#         ToTensorV2(),
#     ]
# )

# import numpy as np
# from glob import glob

# for ft in ft_list:
#     shuffled = info.sample(frac=1, random_state=random_state)
#     train_idx, valid_idx = next(gkf.split(shuffled["FILE"], shuffled[ft], shuffled["EG"]))
#     fold = np.zeros(len(shuffled), dtype=np.int64)
#     fold[valid_idx] = 1
#     shuffled["fold"] = fold

#     train_patches = []
#     train_labels = []
#     for wsi, label in shuffled[shuffled["fold"] == 0][["FILE", ft]].to_numpy():
#         patches = glob(f"{tile_dir}/wsi/*")
#         train_patches += patches
#         train_labels += [label]*len(patches)

#     valid_patches = []
#     valid_labels = []
#     for wsi, label in shuffled[shuffled["fold"] == 1][["FILE", ft]].to_numpy():
#         patches = glob(f"{tile_dir}/wsi/*")
#         valid_patches += patches
#         valid_labels += [label]*len(patches)

#     model = timm.create_model("tf_efficientnet_b4_ns", pretrained=True, num_classes=1)

#     train_dataset = ToxReprCNNDataset(train_patches, train_labels, None, train_transform)
#     valid_dataset = ToxReprCNNDataset(valid_patches, valid_labels, None, valid_transform)

#     train_loader = DataLoader(
#         dataset=train_dataset,
#         num_workers=num_workers,
#         batch_size=train_batch_size,
#         drop_last=True,
#         sampler=ToxReprCNNBalancedSampler(train_dataset, None, np.sum(train_labels)*2),
#     )
#     valid_loader = DataLoader(
#         dataset=valid_dataset,
#         num_workers=num_workers,
#         batch_size=valid_batch_size,
#         shuffle=False,
#     )

#     criterion = nn.BCEWithLogitsLoss()
#     preprocess = lambda x: x.sigmoid()
#     optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#     scheduler = CosineAnnealingLR(optimizer, n_epochs, 5e-7)
#     model = model.to(device)
#     train_history, valid_history, best_score, best_predictions = train_loop(
#         model,
#         train_loader,
#         valid_loader,
#         0,
#         criterion,
#         optimizer,
#         device,
#         n_epochs,
#         scheduler,
#         metrics,
#         f"{save_path}/{ft}",
#         model_name,
#         preprocess,
#         verbose,
#         Logger(),
#     )

#     with open(os.path.join(save_path, f"{save_path}/{ft}/train_result.pickle"), "wb") as f:
#         pickle.dump((train_history, valid_history, best_score, best_predictions), f)